# How to train the Baseline Models

### This notebook will show how to
- instantiate dataloader for the demo data
- instantiate pytorch model
- instantiate a trainer function
- train two baselines on the demo data
- save the model weights (the model weights can already be found in '/notebooks/precomputed_checkpoints/')

### Imports

In [1]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from nnfabrik.builder import get_data, get_model, get_trainer

### Instantiate DataLoader

In [2]:
import os


filenames = ['./data/static26872-17-20-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip',]

dataset_fn = 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': False,
                 'include_eye_position': False,
                 'batch_size': 128,
                 'scale':.25,
                 }

dataloaders = get_data(dataset_fn, dataset_config)

In [3]:
dataloaders

OrderedDict([('train',
              OrderedDict([('26872-17-20',
                            <torch.utils.data.dataloader.DataLoader at 0x7efd67d660d0>)])),
             ('validation',
              OrderedDict([('26872-17-20',
                            <torch.utils.data.dataloader.DataLoader at 0x7efd67d667f0>)])),
             ('test',
              OrderedDict([('26872-17-20',
                            <torch.utils.data.dataloader.DataLoader at 0x7efd67d66760>)])),
             ('final_test',
              OrderedDict([('26872-17-20',
                            <torch.utils.data.dataloader.DataLoader at 0x7efd67d66790>)]))])

In [4]:
b = (next(iter(dataloaders["test"]["26872-17-20"])))

In [5]:
b.images.shape

torch.Size([128, 1, 36, 64])

# Instantiate State of the Art Model (SOTA)

In [6]:
model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
  'layers': 4,
  'input_kern': 9,
  'gamma_input': 6.3831,
  'gamma_readout': 0.0076,
  'hidden_dilation': 1,
  'hidden_kern': 7,
  'hidden_channels': 64,
  'depth_separable': True,
  'grid_mean_predictor': {'type': 'cortex',
   'input_dimensions': 2,
   'hidden_layers': 1,
   'hidden_features': 30,
   'final_tanh': True},
  'init_sigma': 0.1,
  'init_mu_range': 0.3,
  'gauss_type': 'full',
  'shifter': False,
  'stack': -1,
  'use_avg_reg':False,
}

model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

## Configure Trainer

In [ ]:
trainer_fn = "sensorium.training.standard_trainer"

trainer_config = {'max_iter': 200,
                 'verbose': False,
                 'lr_decay_steps': 4,
                 'avg_loss': False,
                 'lr_init': 0.009,
                 }

trainer = get_trainer(trainer_fn=trainer_fn, 
                     trainer_config=trainer_config)

# Run model training

In [ ]:
validation_score, trainer_output, state_dict = trainer(model, dataloaders, seed=42)

# Or load weights

In [7]:
model.load_state_dict(torch.load("./checkpoints/sensorium_sota_model.pth"));
model.eval();

# Now Evaluate

### get submission file

In [8]:
dataset_name = '26872-17-20'

In [9]:
from sensorium import evaluate
# import the API from the competition repo
from sensorium.utility import submission

In [10]:
# generate the submission file
submission.generate_submission_file(trained_model=model, 
                                    dataloaders=dataloaders,
                                    data_key=dataset_name,
                                    path="./sensorium_sota/",
                                    device="cuda")

Submission file saved for tier: test. Saved in: ./sensorium_sota/submission_file_test.csv
Submission file saved for tier: final_test. Saved in: ./sensorium_sota/submission_file_final_test.csv


### get scores

In [11]:
ground_truth_file = './sensorium_ground_truth_file_final_test.csv'
submission_file = './sensorium_sota/submission_file_final_test.csv'

In [12]:
out = evaluate(submission_file, ground_truth_file)

### normal test

In [13]:
print("Results for the SOTA model:")
for metric, value in out.items():
    print(f"{metric}: {np.round(value, 3)}")

Results for the SOTA model:
Single Trial Correlation: 0.287
Average Correlation: 0.528
FEVE: 0.439


### final_test

In [14]:
ground_truth_file = './sensorium_ground_truth_file_test.csv'
submission_file = './sensorium_sota/submission_file_test.csv'

out = evaluate(submission_file, ground_truth_file)

In [15]:
print("Results for the SOTA model:")
for metric, value in out.items():
    print(f"{metric}: {np.round(value, 3)}")

Results for the SOTA model:
Single Trial Correlation: 0.274
Average Correlation: 0.513
FEVE: 0.433


---